<a href="https://colab.research.google.com/github/SamuelEiler/wgan/blob/main/specWGANwganWANDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install wandb -qqq
!pip install --upgrade wandb
!wandb login 50457c76bb245c5d932008de50c1858da49aeeba
import wandb
#wandb.init(project="sWGAN")

from __future__ import division
from matplotlib import pyplot as plt
from torchvision import transforms
from torch.autograd.variable import Variable
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.figure_factory as ff
import torch
import numpy as np
import math


import seaborn as sns
import plotly.express as px
from torch import autograd
from torch.distributions.normal import Normal
import torch.nn.functional as F
from torch import nn
from torch import Tensor
from torch.nn import Parameter

sns.set(rc={'figure.figsize':(11, 8)})

import datetime 
from datetime import date
today = date.today()

import random
import json as js
import pickle
import os
import tqdm.notebook as tq

import torch
import torch.nn as nn
import random
import plotly.graph_objects as go

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

random_seed = 29

np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.set_deterministic(True)

Requirement already up-to-date: wandb in /usr/local/lib/python3.6/dist-packages (0.10.13)
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [22]:
"""Model loading/saving"""
dataPath = './dataset'  


trainCounter = 0
loadVersion = 0
loadModel = False
epoch_loaded = 0
path = './SineGAN/' + str(trainCounter)

loadPath = '/content/epoch500.tar'


#loadPath = '/content/SineGAN/0/epoch148.tar'


"""Defining parameters"""
noise_features = 3 #frequency, amplitude, shift
sample_size = 64 * 2 #batch size needed for Data Loader and the noise creator function.


#Params for the generator
hidden_nodes_g = 50
layers = 1
tanh_layer = False
bidir = False

dropout = 0.05

#No. of training rounds per epoch
D_rounds = 1
G_rounds = 1
num_epoch = 250
#learning_rate = 0.0001 # 0.00005  * 200 #*2 for adam 
lr_decrease_factor = 0.98
lr_start_factor = 10
lr_g = 0.00001 #* 0.5
lr_d = 0.001 #* 0.5
beta1 = 0.0
beta2 = 0.999
opt_adam = True #working: adam lr*2 lp3
opt = ''
if opt_adam:
  opt = 'Adam'
else:
  opt = 'RMS'
#learning_rate=0.0002, beta_1=0.5, beta_2=0.9

lipschitz_constraint = 10
center = 1
onesided = True
penalty_weight = 10
if onesided:
    clip_fn = lambda x: x.clamp(max=0)
else:
    clip_fn = lambda x: x    
#Params for the Discriminator
minibatch_layer = 0
minibatch_normal_init_ = True
num_cvs = 2
cv1_out= 5
cv1_k = 3
cv1_s = 2
p1_k = 3
p1_s = 2
cv2_out = 10
cv2_k = 5
cv2_s = 2
p2_k = 3
p2_s = 2

wandb.init(project="sWGAN", config={
    'center_gp': center,
    'dropout': dropout,
    'random_seed': random_seed,
    "lipschitz_constraint": lipschitz_constraint,
    "D_rounds": D_rounds,
    "optim": opt,
    'beta1': beta1,
    'beta2': beta2,
    'penalty_weight': penalty_weight,
    'one_side': onesided,
    'sample_size' : sample_size, 
    'noise_features' : noise_features,
    'seq_length' : 50,
    'num_layers': layers, 
    'bidir': bidir,
    'hidden-initialization': 'zeros',
    'hidden_dims_generator': hidden_nodes_g, 
    'minibatch_layer': minibatch_layer,
    'minibatch_normal_init_' : minibatch_normal_init_,
    'num_cvs':num_cvs,
    'cv1_out':cv1_out,
    'cv1_k':cv1_k,
    'cv1_s':cv1_s,
    'p1_k':p1_k,
    'p1_s':p1_s,
    'cv2_out':cv2_out,
    'cv2_k':cv2_k,
    'cv2_s':cv2_s,
    'p2_k':p2_k,
    'p2_s':p2_s,
    'num_epoch':num_epoch,
    'D_rounds': D_rounds,
    'G_rounds': G_rounds,  
    'D_lr' : lr_d,
    'G_lr' : lr_g,
    'lr_decrease_factor': lr_decrease_factor,
    'lr_start_factor': lr_start_factor


})
config = wandb.config


lr_g = lr_g * lr_start_factor
lr_d = lr_d * lr_start_factor

Critic,-1.84763
Generator,3.98805
GradientPenalty,0.21467
_step,10
_runtime,67
_timestamp,1610623941


Critic,█▇▁
Generator,█▃▁
GradientPenalty,▂█▁
_step,▁▂▂▃▄▅▅▆▇▇█
_runtime,▁▁▁▁▃▃▃▅▅▅█
_timestamp,▁▁▁▁▃▃▃▅▅▅█


In [23]:
ecg = 0
import numpy as np

if ecg:

  from google.colab import drive
  drive.mount('/content/drive')

  !pip install arff2pandas

  from arff2pandas import a2p

  path = '/content/drive/MyDrive/MasterThesis/ECG_Dataset/ECG5000/'
  with open(path+'ECG5000_TRAIN.arff') as f:
    train = a2p.load(f)
  with open(path+'ECG5000_TEST.arff') as f:
    test = a2p.load(f)

  df = train.append(test)
  df = df.sample(frac=1.0)
  df.shape

  CLASS_NORMAL = 1
  class_names = ['Normal','R on T','PVC','SP','UB']

  new_columns = list(df.columns)
  new_columns[-1] = 'target'
  df.columns = new_columns

  df.target.value_counts()

  normal_df = df[df.target == str(CLASS_NORMAL)].drop(labels='target', axis=1)
  print(normal_df.shape)

  anomaly_df = df[df.target != str(CLASS_NORMAL)].drop(labels='target', axis=1)
  print(anomaly_df.shape)

  from sklearn.model_selection import train_test_split
  RANDOM_SEED = 29
  np.random.seed(RANDOM_SEED)
  torch.manual_seed(RANDOM_SEED)


  train_df, val_df = train_test_split(
    normal_df,
    test_size=0.15,
    random_state=RANDOM_SEED
  )

  data = pd.DataFrame(train_df[1:][:])  

  dataPath = './dataset'  
  if not os.path.exists(dataPath):
    os.makedirs(dataPath)

  data.to_csv(dataPath + '/ecg_data_v1.csv', header = False, index = False)


  val_df, test_df = train_test_split(
    val_df,
    test_size=0.33,
    random_state=RANDOM_SEED
  )

  def create_dataset(df):
    sequences = df.astype(np.float32).to_numpy().tolist()
    dataset = [torch.tensor(s).unsqueeze(1).float() for s in sequences]
    n_seq, seq_len, n_features = torch.stack(dataset).shape
    return dataset, seq_len, n_features


  train_dataset, seq_len, n_features = create_dataset(train_df)
  val_dataset, _, _ = create_dataset(val_df)
  test_normal_dataset, _, _ = create_dataset(test_df)
  test_anomaly_dataset, _, _ = create_dataset(anomaly_df)

  import torch
  from torch.utils.data import Dataset, DataLoader
  import pandas as pd

  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  class ECGData(Dataset):
    #This is the class for teh ECG Data that we need to load, transform and then use in teh dataloader.
    def __init__(self,source_file,transform = None):
      self.source_file = source_file
      self.data  = pd.read_csv(source_file, header = None)
      self.transform = transform
      
    def __len__(self):
      return self.data.shape[0]
      
    def __getitem__(self,idx):
      
      sample = self.data.iloc[idx]
      
      if self.transform:
          sample = self.transform(sample)
          
      return sample   

  """Including the function that will transform the dataframe to a pytorch tensor"""
  class PD_to_Tensor(object):
      def __call__(self,sample):
        return torch.tensor(sample.values).to(device=device, dtype=float)


  print(len(train_dataset))
  print(seq_len)
  print(n_features)
  for n in range(10):
    plt.plot(train_dataset[n])
  plt.show()

  def GetECGData(source_file):
    compose = transforms.Compose([PD_to_Tensor()])
    return ECGData(source_file ,transform = compose)





In [24]:
import torch
import torch.nn.functional as F

#define _l2normalization
def _l2normalize(v, eps=1e-12):
    return v / (torch.norm(v) + eps)

def max_singular_value(W, u=None, Ip=1):
    """
    power iteration for weight parameter
    """
    #xp = W.data
    if not Ip >= 1:
        raise ValueError("Power iteration should be a positive integer")
    if u is None:
        u = torch.FloatTensor(1, W.size(0)).normal_(0, 1).cuda()
    _u = u
    for _ in range(Ip):
        _v = _l2normalize(torch.matmul(_u, W.data), eps=1e-12)
        _u = _l2normalize(torch.matmul(_v, torch.transpose(W.data, 0, 1)), eps=1e-12)
    sigma = torch.sum(F.linear(_u, torch.transpose(W.data, 0, 1)) * _v)
    return sigma, _u



import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import Linear

class SNLinear(Linear):
    r"""Applies a linear transformation to the incoming data: :math:`y = Ax + b`
       Args:
           in_features: size of each input sample
           out_features: size of each output sample
           bias: If set to False, the layer will not learn an additive bias.
               Default: ``True``
       Shape:
           - Input: :math:`(N, *, in\_features)` where :math:`*` means any number of
             additional dimensions
           - Output: :math:`(N, *, out\_features)` where all but the last dimension
             are the same shape as the input.
       Attributes:
           weight: the learnable weights of the module of shape
               `(out_features x in_features)`
           bias:   the learnable bias of the module of shape `(out_features)`
           W(Tensor): Spectrally normalized weight
           u (Tensor): the right largest singular value of W.
       """
    def __init__(self, in_features, out_features, bias=True):
        super(SNLinear, self).__init__(in_features, out_features, bias)
        self.register_buffer('u', torch.Tensor(1, out_features).normal_())

    @property
    def W_(self):
        w_mat = self.weight.view(self.weight.size(0), -1)
        sigma, _u = max_singular_value(w_mat, self.u)
        self.u.copy_(_u)
        return self.weight / sigma

    def forward(self, input):
        return F.linear(input, self.W_, self.bias)



import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import Conv1d
from torch.nn.modules.utils import _pair

class SNConv1d(Conv1d):

    r"""Applies a 2D convolution over an input signal composed of several input
    planes.
    In the simplest case, the output value of the layer with input size
    :math:`(N, C_{in}, H, W)` and output :math:`(N, C_{out}, H_{out}, W_{out})`
    can be precisely described as:
    .. math::
        \begin{array}{ll}
        out(N_i, C_{out_j})  = bias(C_{out_j})
                       + \sum_{{k}=0}^{C_{in}-1} weight(C_{out_j}, k)  \star input(N_i, k)
        \end{array}
    where :math:`\star` is the valid 2D `cross-correlation`_ operator,
    :math:`N` is a batch size, :math:`C` denotes a number of channels,
    :math:`H` is a height of input planes in pixels, and :math:`W` is
    width in pixels.
    | :attr:`stride` controls the stride for the cross-correlation, a single
      number or a tuple.
    | :attr:`padding` controls the amount of implicit zero-paddings on both
    |  sides for :attr:`padding` number of points for each dimension.
    | :attr:`dilation` controls the spacing between the kernel points; also
      known as the à trous algorithm. It is harder to describe, but this `link`_
      has a nice visualization of what :attr:`dilation` does.
    | :attr:`groups` controls the connections between inputs and outputs.
      `in_channels` and `out_channels` must both be divisible by `groups`.
    |       At groups=1, all inputs are convolved to all outputs.
    |       At groups=2, the operation becomes equivalent to having two conv
                 layers side by side, each seeing half the input channels,
                 and producing half the output channels, and both subsequently
                 concatenated.
            At groups=`in_channels`, each input channel is convolved with its
                 own set of filters (of size `out_channels // in_channels`).
    The parameters :attr:`kernel_size`, :attr:`stride`, :attr:`padding`, :attr:`dilation` can either be:
        - a single ``int`` -- in which case the same value is used for the height and width dimension
        - a ``tuple`` of two ints -- in which case, the first `int` is used for the height dimension,
          and the second `int` for the width dimension
    .. note::
         Depending of the size of your kernel, several (of the last)
         columns of the input might be lost, because it is a valid `cross-correlation`_,
         and not a full `cross-correlation`_.
         It is up to the user to add proper padding.
    .. note::
         The configuration when `groups == in_channels` and `out_channels = K * in_channels`
         where `K` is a positive integer is termed in literature as depthwise convolution.
         In other words, for an input of size :math:`(N, C_{in}, H_{in}, W_{in})`, if you want a
         depthwise convolution with a depthwise multiplier `K`,
         then you use the constructor arguments
         :math:`(in\_channels=C_{in}, out\_channels=C_{in} * K, ..., groups=C_{in})`
    Args:
        in_channels (int): Number of channels in the input image
        out_channels (int): Number of channels produced by the convolution
        kernel_size (int or tuple): Size of the convolving kernel
        stride (int or tuple, optional): Stride of the convolution. Default: 1
        padding (int or tuple, optional): Zero-padding added to both sides of the input. Default: 0
        dilation (int or tuple, optional): Spacing between kernel elements. Default: 1
        groups (int, optional): Number of blocked connections from input channels to output channels. Default: 1
        bias (bool, optional): If ``True``, adds a learnable bias to the output. Default: ``True``
    Shape:
        - Input: :math:`(N, C_{in}, H_{in}, W_{in})`
        - Output: :math:`(N, C_{out}, H_{out}, W_{out})` where
          :math:`H_{out} = floor((H_{in}  + 2 * padding[0] - dilation[0] * (kernel\_size[0] - 1) - 1) / stride[0] + 1)`
          :math:`W_{out} = floor((W_{in}  + 2 * padding[1] - dilation[1] * (kernel\_size[1] - 1) - 1) / stride[1] + 1)`
    Attributes:
        weight (Tensor): the learnable weights of the module of shape
                         (out_channels, in_channels, kernel_size[0], kernel_size[1])
        bias (Tensor):   the learnable bias of the module of shape (out_channels)
        W(Tensor): Spectrally normalized weight
        u (Tensor): the right largest singular value of W.
    .. _cross-correlation:
        https://en.wikipedia.org/wiki/Cross-correlation
    .. _link:
        https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md
    """

#padding_mode: str='zeros'

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        kernel_size = (kernel_size)
        stride = (stride)
        padding = (padding)
        dilation = (dilation)
        super(SNConv1d, self).__init__(
            in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias)
        self.register_buffer('u', torch.Tensor(1, out_channels).normal_())

    @property
    def W_(self):
        w_mat = self.weight.view(self.weight.size(0), -1)
        sigma, _u = max_singular_value(w_mat, self.u)
        self.u.copy_(_u)
        return self.weight / sigma

    def forward(self, input):
        return F.conv1d(input, self.W_, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)
        


### Models (Generator and Discriminator)

In [25]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def l2normalize(v, eps=1e-12):
    return v / (v.norm() + eps)


class SpectralNorm(nn.Module):
    def __init__(self, module, name='weight', power_iterations=1):
        super(SpectralNorm, self).__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        if not self._made_params():
            self._make_params()

    def _update_u_v(self):
        u = getattr(self.module, self.name + "_u")
        v = getattr(self.module, self.name + "_v")
        w = getattr(self.module, self.name + "_bar")

        height = w.data.shape[0]
        for _ in range(self.power_iterations):
            v.data = l2normalize(torch.mv(torch.t(w.view(height,-1).data), u.data))
            u.data = l2normalize(torch.mv(w.view(height,-1).data, v.data))

        # sigma = torch.dot(u.data, torch.mv(w.view(height,-1).data, v.data))
        sigma = u.dot(w.view(height, -1).mv(v))
        setattr(self.module, self.name, w / sigma.expand_as(w))

    def _made_params(self):
        try:
            u = getattr(self.module, self.name + "_u")
            v = getattr(self.module, self.name + "_v")
            w = getattr(self.module, self.name + "_bar")
            return True
        except AttributeError:
            return False


    def _make_params(self):
        w = getattr(self.module, self.name)

        height = w.data.shape[0]
        width = w.view(height, -1).data.shape[1]

        u = Parameter(w.data.new(height).normal_(0, 1), requires_grad=False)
        v = Parameter(w.data.new(width).normal_(0, 1), requires_grad=False)
        u.data = l2normalize(u.data)
        v.data = l2normalize(v.data)
        w_bar = Parameter(w.data)

        del self.module._parameters[self.name]

        self.module.register_parameter(self.name + "_u", u)
        self.module.register_parameter(self.name + "_v", v)
        self.module.register_parameter(self.name + "_bar", w_bar)


    def forward(self, *args):
        self._update_u_v()
        return self.module.forward(*args)

class MinibatchDiscrimination(torch.nn.Module):
   def __init__(self,input_features,output_features,minibatch_normal_init, hidden_features=16):
      super(MinibatchDiscrimination,self).__init__()
      
      self.input_features = input_features
      self.output_features = output_features
      self.hidden_features = hidden_features
      self.T = torch.nn.Parameter(torch.randn(self.input_features,self.output_features, self.hidden_features))
      if minibatch_normal_init == True:
        nn.init.normal_(self.T, 0,1)
      
   def forward(self,x):
      M = torch.mm(x,self.T.view(self.input_features,-1))
      M = M.view(-1, self.output_features, self.hidden_features).unsqueeze(0)
      M_t = M.permute(1, 0, 2, 3)
      # Broadcasting reduces the matrix subtraction to the form desired in the paper
      out = torch.sum(torch.exp(-(torch.abs(M - M_t).sum(3))), dim=0) - 1
      
      return torch.cat([x, out], 1)
    

# Use minibatch = 0 for no minibatch discriminiation layer to be used in the architecture. If minibatch > 0, then minibatch is the number of output dimensions of the MBD layer.
class Discriminator(torch.nn.Module):
  def __init__(self,seq_length,batch_size,minibatch_normal_init, n_features = 1, num_cv = 1, minibatch = 0, cv1_out= 10, cv1_k = 3, cv1_s = 4, p1_k = 3, p1_s = 3, cv2_out = 10, cv2_k = 3, cv2_s = 3 ,p2_k = 3, p2_s = 3):
      super(Discriminator,self).__init__()
      self.n_features = n_features
      self.seq_length = seq_length
      self.batch_size = batch_size
      self.num_cv = num_cv
      self.minibatch = minibatch
      self.cv1_dims = int((((((seq_length - cv1_k)/cv1_s) + 1)-p1_k)/p1_s)+1)
      self.cv2_dims = int((((((self.cv1_dims - cv2_k)/cv2_s) + 1)-p2_k)/p2_s)+1)
      self.cv1_out = cv1_out
      self.cv2_out = cv2_out
      

      self.dropout = torch.nn.Dropout(0.05)
      conv1_features = self.n_features


      self.conv1 = SNConv1d(conv1_features, cv1_out, cv1_k, cv1_s, 1, 1, 1 , True)
      self.conv2 = SNConv1d(cv1_out, cv2_out, cv2_k, cv1_s, 1, 1, 1 ,True)

      self.out1 = SNLinear(cv2_out*12, cv2_out*12)
      self.out2 = SNLinear(cv2_out*12, 1)

      self.lr = nn.LeakyReLU(0.1, inplace=True)


      self.cv1 = SpectralNorm(nn.Conv1d(conv1_features, cv1_out, cv1_k, cv1_s,1))
      self.cv2 = SpectralNorm(nn.Conv1d(cv1_out, cv2_out, cv2_k, cv2_s,1))
      self.o1 = SpectralNorm(nn.Linear(cv2_out*12, cv2_out*12))
      self.o2 = SpectralNorm(nn.Linear(cv2_out*12, 1))
      self.l = nn.LeakyReLU(0.1)


  def forward(self,x):
      self.batch_size = x.size(0)
      x = x.view(self.batch_size,1,self.seq_length) 

      x = self.l(self.cv1(x))
      x = self.l(self.cv2(x))
      x = x.view(self.batch_size, -1)
      x = self.l(self.o1(x))
      x = self.dropout(x)
      x = self.o2(x)
      return x


      #print(f'in {x.shape}')
      x = self.lr(self.conv1(x))
      #print(f'aa {x.shape}')
      x = self.lr(self.conv2(x))
      #print(f'ab {x.shape}')
      #print(f'abp {x.shape}')
      x = x.view(self.batch_size, -1)
   #  print(f'conv2 out {x.shape}') #64, 176

      x = self.lr(self.out1(x))
      x = self.dropout(x)
      #print(f'la {x.shape}')
      x = self.out2(x)
      #print(f'lb {x.shape}')
      return x
      

class DLSTM(torch.nn.Module):
  #seq_length not important
  def __init__(self):
      super(DLSTM,self).__init__()
      self.n_features = 1
      self.hidden_dim = 50
      self.num_layers = 2
      self.seq_length = 50
      self.batch_size = 64
      self.bidirectional = False

      #self.dropout = torch.nn.Dropout(dropout)

      
      #Checking if the architecture uses a BiLSTM and setting the output parameters as appropriate.
      if self.bidirectional == True:
        self.num_dirs = 2
      else:
        self.num_dirs = 1
      
      
      self.layer1 = torch.nn.LSTM(input_size = self.n_features, hidden_size = self.hidden_dim, num_layers = self.num_layers,batch_first = True, bidirectional = self.bidirectional )
      self.out = torch.nn.Linear(self.hidden_dim,50) # to make sure the output is between 0 and 1 - removed ,torch.nn.Sigmoid()
  
  def forward(self,x):
      self.seq_length = x.size(1)
      self.batch_size = x.size(0)

      h_0 = torch.zeros(self.num_layers*self.num_dirs, self.batch_size, self.hidden_dim, requires_grad=True).to(device)
      c_0 = torch.zeros(self.num_layers*self.num_dirs, self.batch_size, self.hidden_dim, requires_grad=True).to(device)
      #x, _ = self.layer1(x.view(self.batch_size,self.seq_length,1), (h_0, c_0)) 
      x, _ = self.layer1(x.view(self.batch_size,self.seq_length, self.n_features), (h_0, c_0)) #--------------------------------noise_features ---------------------------------------


      if self.bidirectional == True:
        x = x.view(x.size(0), x.size(1), 2, -1).sum(2).view(x.size(0), x.size(1), -1)
      
      #Note that the output of the bidirectional LSTM is in the form (batch_size,seq_lenth,num_dirs*hidden_dim) To separate the directions, we can use 
      #x.view(self.batch_size,self.seq_length,self.num_dirs, self.hidden_dim)
      #x = self.dropout(x)

      x = self.out(x)

      return x.squeeze() #,hidden

class Generator(torch.nn.Module):
  #seq_length not important
  def __init__(self,seq_length,batch_size,n_features = 1, hidden_dim = 50, num_layers = 2, tanh_output = False, bidirectional = False):
      super(Generator,self).__init__()
      self.n_features = n_features
      self.hidden_dim = hidden_dim
      self.num_layers = num_layers
      self.seq_length = seq_length
      self.batch_size = batch_size
      self.tanh_output = tanh_output
      self.bidirectional = bidirectional

      #self.dropout = torch.nn.Dropout(dropout)

      
      #Checking if the architecture uses a BiLSTM and setting the output parameters as appropriate.
      if self.bidirectional == True:
        self.num_dirs = 2
      else:
        self.num_dirs = 1
      
      
      self.layer1 = torch.nn.LSTM(input_size = self.n_features, hidden_size = self.hidden_dim, num_layers = self.num_layers,batch_first = True, bidirectional = self.bidirectional )
      self.out = torch.nn.Linear(self.hidden_dim,1) # to make sure the output is between 0 and 1 - removed ,torch.nn.Sigmoid()
  
  def forward(self,x):
      self.seq_length = x.size(1)
      self.batch_size = x.size(0)

      h_0 = torch.zeros(self.num_layers*self.num_dirs, self.batch_size, self.hidden_dim, requires_grad=True).to(device)
      c_0 = torch.zeros(self.num_layers*self.num_dirs, self.batch_size, self.hidden_dim, requires_grad=True).to(device)
      #x, _ = self.layer1(x.view(self.batch_size,self.seq_length,1), (h_0, c_0)) 
      x, _ = self.layer1(x.view(self.batch_size,self.seq_length, self.n_features), (h_0, c_0)) #--------------------------------noise_features ---------------------------------------


      if self.bidirectional == True:
        x = x.view(x.size(0), x.size(1), 2, -1).sum(2).view(x.size(0), x.size(1), -1)
      
      #Note that the output of the bidirectional LSTM is in the form (batch_size,seq_lenth,num_dirs*hidden_dim) To separate the directions, we can use 
      #x.view(self.batch_size,self.seq_length,self.num_dirs, self.hidden_dim)
      #x = self.dropout(x)

      x = self.out(x)

      return x.squeeze() #,hidden

### Data Loader

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class SineData(Dataset):
  #This is the class for teh ECG Data that we need to load, transform and then use in teh dataloader.
  def __init__(self,source_file,transform = None):
    self.source_file = source_file
    self.data  = pd.read_csv(source_file, header = None)
    self.transform = transform
    
  def __len__(self):
    return self.data.shape[0]
    
  def __getitem__(self,idx):
    
    sample = self.data.iloc[idx]
    
    if self.transform:
        sample = self.transform(sample)
        
    return sample   

"""Including the function that will transform the dataframe to a pytorch tensor"""
class PD_to_Tensor(object):
    def __call__(self,sample):
      return torch.tensor(sample.values).to(device=device, dtype=float)

### Create Dataset

In [27]:
import pandas as pd
import numpy as np
import random
import os

dataPath = './dataset'  
if not os.path.exists(dataPath):
        os.makedirs(dataPath)

trainingset_size = 64 * 150
testset_size = 64 * 20

"""Create a training set of sine waves with 10000 records"""

#a = np.arange(0.1,0.9,0.02)
x = np.arange(0,20,0.4)
#r = np.arange(2,6.1,0.1)
count = 0
fs = len(x)
y = np.zeros((1,len(x)))

for n in range(trainingset_size):
  amp = random.uniform(0.1, 0.9)
  rad = random.uniform(5, 10)
  phase = random.uniform(-fs/0.5*np.pi,fs/0.5*np.pi)
  offsetY = 0 #random.uniform(-1, 1)
  y = np.append(y,  offsetY + amp*np.sin(((2*np.pi*rad*x)+phase)/fs).reshape((1,len(x))),axis = 0)
  
# for n in range(20):
#    plt.plot(y[n + 1])
# plt.show()
data = pd.DataFrame(y[1:][:])  
data.to_csv(dataPath + '/sinedata_v2.csv', header = False, index = False)

"""Creating a test set of sine waves with 3000 records"""
a = np.arange(0.1,0.9,0.02)
x = np.arange(0,20,0.5)
r = np.arange(2,6.1,0.1)
count = 0
fs = len(x)
y = np.zeros((1,len(x)))

for n in range(testset_size):
  amp = a[random.randint(0,len(a)-1)]
  rad = r[random.randint(0,len(r)-1)]
  phase = random.uniform(-1,1)*np.pi

  y = np.append(y,amp*np.sin(((2*np.pi*rad*x)+phase)/fs).reshape((1,len(x))),axis = 0)
  
data = pd.DataFrame(y[1:][:])  
data.to_csv(dataPath + '/sinedata_test_v2.csv', header = False, index = False)

### Utils

In [28]:
def noise(batchsize, seq_length, noise_features):
    rnoise = torch.ones( (batchsize, seq_length, noise_features)).to(device)
    if noise_features == 1:
      return torch.rand(batchsize, seq_length, 1)

    for b in range(batchsize):
        randoms = np.ones(noise_features)
        for nf in range(noise_features):
            randoms[nf] = random.uniform(-1,1)
        for i in range(seq_length):
            for nf in range(noise_features):
                rnoise[b][i][nf] = randoms[nf]
    return rnoise




def pdist(sample_1, sample_2, norm=2, eps=1e-5):
    n_1, n_2 = sample_1.size(0), sample_2.size(0)
    norm = float(norm)
    
    if norm == 2.:
        norms_1 = torch.sum(sample_1**2, dim=1, keepdim=True)
        norms_2 = torch.sum(sample_2**2, dim=1, keepdim=True)
        norms = (norms_1.expand(n_1, n_2) +
                 norms_2.transpose(0, 1).expand(n_1, n_2))
        distances_squared = norms - 2 * sample_1.mm(sample_2.t())
        return torch.sqrt(eps + torch.abs(distances_squared))
    else:
        dim = sample_1.size(1)
        expanded_1 = sample_1.unsqueeze(1).expand(n_1, n_2, dim)
        expanded_2 = sample_2.unsqueeze(0).expand(n_1, n_2, dim)
        differences = torch.abs(expanded_1 - expanded_2) ** norm
        inner = torch.sum(differences, dim=2, keepdim=False)
        return (eps + inner) ** (1. / norm)

def permutation_test_mat(matrix,n_1,  n_2,  n_permutations, a00=1,  a11=1,  a01=0):
    n = n_1 + n_2
    pi = np.zeros(n, dtype=np.int8)
    pi[n_1:] = 1

    larger = 0.
    count = 0
    
    for sample_n in range(1 + n_permutations):
        count = 0.
        for i in range(n):
            for j in range(i, n):
                mij = matrix[i, j] + matrix[j, i]
                if pi[i] == pi[j] == 0:
                    count += a00 * mij
                elif pi[i] == pi[j] == 1:
                    count += a11 * mij
                else:
                    count += a01 * mij
        if sample_n == 0:
            statistic = count
        elif statistic <= count:
            larger += 1

        np.random.shuffle(pi)

    return larger / n_permutations


class MMDStatistic:
    def __init__(self, n_1, n_2):
        self.n_1 = n_1
        self.n_2 = n_2

        # The three constants used in the test.
        self.a00 = 1. / (n_1 * (n_1 - 1))
        self.a11 = 1. / (n_2 * (n_2 - 1))
        self.a01 = - 1. / (n_1 * n_2)

    def __call__(self, sample_1, sample_2, alphas, ret_matrix=False):

        sample_12 = torch.cat((sample_1, sample_2), 0)
        distances = pdist(sample_12, sample_12, norm=2)

        kernels = None
        for alpha in alphas:
            kernels_a = torch.exp(- alpha * distances ** 2)
            if kernels is None:
                kernels = kernels_a
            else:
                kernels = kernels + kernels_a

        k_1 = kernels[:self.n_1, :self.n_1]
        k_2 = kernels[self.n_1:, self.n_1:]
        k_12 = kernels[:self.n_1, self.n_1:]

        mmd = (2 * self.a01 * k_12.sum() +
               self.a00 * (k_1.sum() - torch.trace(k_1)) +
               self.a11 * (k_2.sum() - torch.trace(k_2)))
        if ret_matrix:
            return mmd, kernels
        else:
            return mmd


    def pval(self, distances, n_permutations=1000):
        if isinstance(distances, Variable):
            distances = distances.data
        return permutation_test_mat(distances.cpu().numpy(),
                                    self.n_1, self.n_2,
                                    n_permutations,
                                    a00=self.a00, a11=self.a11, a01=self.a01)



def saveModel(path, epoch): 
    
    torch.save({
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'optimizerG_state_dict': generator_opt.state_dict(),
            'optimizerD_state_dict': discriminator_opt.state_dict(),  
            'D_losses': D_losses,
            'G_losses': G_losses,
            'mmd_list': mmd_list,
            'gradient_p': gradient_p,
            'series_list': series_list,
            'epoch': epoch
            }, path +'/epoch'+str(epoch)+'.tar')


def pairwisedistances(X,Y,norm=2):
    dist = pdist(X,Y,norm)
    return np.median(dist.numpy())


def GetSineData(source_file):
  compose = transforms.Compose([PD_to_Tensor()])
  return SineData(source_file ,transform = compose)

def compute_grad2(d_out, x_in):
    batch_size = x_in.size(0)
    grad_dout = autograd.grad(
        outputs=d_out.sum(), inputs=x_in,
        create_graph=True, retain_graph=True, only_inputs=True
    )[0]
    grad_dout2 = grad_dout.pow(2)
    assert(grad_dout2.size() == x_in.size())
    reg = grad_dout2.view(batch_size, -1).sum(1)
    return reg

def wgan_gp_reg(discriminator, x_real, x_fake, center=0.):
    batch_size = x_fake.size(0)
    #x_real = x_real.unsqueeze(1)
    #x_fake = x_fake.unsqueeze(1)
    #print(f'xrea {x_real.shape}')
    #print(f'xfake {x_fake.shape}')
    eps = torch.rand(batch_size, device=x_fake.device).view(batch_size, 1)
    x_interp = (1 - eps) * x_real + eps * x_fake
    x_interp = x_interp.detach()
    x_interp.requires_grad_()
    #print(f'intr {x_interp.shape}')
    d_out = discriminator(x_interp)

    reg = ( torch.sqrt(compute_grad2(d_out, x_interp) + 1e-12) - center).pow(2).mean()

    #print(f'reg: {reg}')
    return reg



### Variables

In [29]:


        # if lipschitz_constraint == 0:
        #     # Calculate interpolation
        #     epsilon = torch.rand(sample_size, 1).to(device)
        #     epsilon = epsilon.expand_as(real_data)
        #     interpolated = epsilon * real_data + (1 - epsilon) * fake_data # interpolates := xhat        
        #     interpolated = Variable(interpolated, requires_grad=True).to(device)
        #     # Calculate probability of interpolated examples
        #     prob_interpolated = discriminator(interpolated)
        #     gradients = torch.autograd.grad(outputs=prob_interpolated, inputs=interpolated, grad_outputs=torch.ones_like(prob_interpolated), create_graph=True, retain_graph=True)[0]
        #     # set up tensors (torch administrativa)
        #     # Gradients have shape (batch_size, num_channels, img_width, img_height),
        #     # so flatten to easily take norm per example in batch
        #     gradients = gradients.view(sample_size, -1)
        #     # Derivatives of the gradient close to 0 can cause problems because of
        #     # the square root, so manually calculate norm and add epsilon
        #     gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)
        #     gradient_penalty =penalty_weight * ((gradients_norm - 1) ** 2).mean()        
        #     loss_discriminator = loss_discriminator +   gradient_penalty
        
        # elif lipschitz_constraint == 1:
        #         dist = ((real_data-fake_data)**2).sum(1)**0.5
        #         lip_est = (critic_out_real.mean()-critic_out_fake.mean()).abs()/(dist+1e-8)
        #         lip_loss = penalty_weight*(clip_fn(1.0-lip_est)**2).mean(0).view(1)
        #         gradient_penalty = lip_loss
        #         loss_discriminator = loss_discriminator + lip_loss
        # elif lipschitz_constraint == 3:           
        #         interp_alpha.resize_(sample_size, 1)
        #         interp_alpha.uniform_()
        #         interp_points = Variable((interp_alpha.expand_as(real_data.data)*real_data.data+(1-interp_alpha.expand_as(real_data.data))*fake_data.data), requires_grad=True)
        #         errD_interp_vec = discriminator(interp_points)
        #         errD_gradient, = torch.autograd.grad(errD_interp_vec.sum(), interp_points, create_graph=True)
        #         lip_est = (errD_gradient**2).view(sample_size,-1).sum(1)**0.5 # updated: bug fix: added **0.5
        #         lip_loss = penalty_weight*(clip_fn(1.0-lip_est)**2).mean(0).view(1)
        #         gradient_penalty = lip_loss
        #         loss_discriminator = loss_discriminator + lip_loss
        # elif lipschitz_constraint == 4 or lipschitz_constraint == 5:
        #     if lipschitz_constraint == 4:
        #       # this tries to match DRAGAN
        #       perturbation.resize_as_(real_data.data)
        #       perturbation.uniform_()
        #       perturbation *= 0.5*real_data.data.std()
        #       interp_alpha.resize_(sample_size, 1)  
        #       interp_alpha.uniform_()
        #       perturbation *= interp_alpha.expand_as(perturbation)
        #     else:
        #       # the choice of perturbation and leaving out alpha differs from the DRAGAN article
        #       perturbation.resize_as_(real_data.data)
        #       perturbation.normal_()
        #       perturbation *= 0.25*real_data.data.std(0).expand_as(perturbation)


        #     perturbation += real_data.data
        #     interp_points = Variable(perturbation, requires_grad=True)
        #     errD_interp_vec = discriminator(interp_points)
        #     errD_gradient, = torch.autograd.grad(errD_interp_vec.sum(), interp_points, create_graph=True)
        #     lip_est = (errD_gradient**2).view(sample_size,-1).sum(1)**0.5
        #     lip_loss = penalty_weight*(clip_fn(1.0-lip_est)**2).mean(0).view(1)
        #     gradient_penalty = lip_loss
        #     loss_discriminator = loss_discriminator + lip_loss


### Training

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print(f'Using : {device}')
    print(torch.cuda.get_device_name(device))
else :
    print(f'Using : {device}')





"""Creating the training set of sine signals"""



source_filename =  dataPath + '/sinedata_v2.csv'
sine_data = GetSineData(source_file = source_filename)
data_loader = torch.utils.data.DataLoader(sine_data, batch_size=sample_size, shuffle=True, drop_last=True)
seq_length = sine_data[0].size()[0] #Number of features

if ecg:
  ecg_path =  dataPath + '/ecg_data_v1.csv'
  sine_data = GetECGData(source_file = ecg_path)
  data_loader = torch.utils.data.DataLoader(sine_data, batch_size=sample_size, shuffle=True)
  seq_length = 140






# Num batches
num_batches = len(data_loader)

"""Creating the Test Set"""
test_filename = dataPath + '/sinedata_test_v2.csv' #'./sinedata_test_v2.csv'

sine_data_test = GetSineData(source_file = test_filename)
data_loader_test = torch.utils.data.DataLoader(sine_data_test, batch_size=sample_size, shuffle=True)


""" Evaluation of GAN with 1 CNN Layer in Discriminator
##Generator and Discriminator training phase
"""
#minibatch_out = [5] #[0,3,5,8,10]
#for minibatch_layer in tq.tqdm(minibatch_out):
#path = "./output/Run_"+str(today.strftime("%d_%m_%Y"))+"_"+ str(datetime.datetime.now().time()).split('.')[0]


if not os.path.exists(path):
        os.makedirs(path)




#Initialising the generator and discriminator
generator = Generator(seq_length,sample_size,n_features = noise_features, hidden_dim =  hidden_nodes_g,num_layers= layers, tanh_output = tanh_layer, bidirectional = bidir).to(device)
discriminator = Discriminator(seq_length, sample_size ,minibatch_normal_init = minibatch_normal_init_, minibatch = minibatch_layer,num_cv = num_cvs, cv1_out = cv1_out,cv1_k = cv1_k, cv1_s = cv1_s, p1_k = p1_k, p1_s = p1_s, cv2_out= cv2_out, cv2_k = cv2_k, cv2_s = cv2_s, p2_k = p2_k, p2_s = p2_s).to(device)
#discriminator = DLSTM()

#Defining optimizer
if opt_adam:
    discriminator_opt = torch.optim.Adam(  filter(lambda p: p.requires_grad, discriminator.parameters())   ,lr = lr_d, betas=[beta1, beta2])
    generator_opt = torch.optim.Adam(generator.parameters(),lr = lr_g,  betas=[beta1, beta2])
else:
    discriminator_opt = torch.optim.RMSprop(discriminator.parameters(), lr=learning_rate)
    generator_opt = torch.optim.RMSprop(generator.parameters(), lr=learning_rate)

gradient_p = []
G_losses = []
D_losses = []
mmd_list = []
series_list = np.zeros((1,seq_length))


if loadModel:
    checkpoint = torch.load(loadPath)#torch.load(loadPath)
    generator.load_state_dict(checkpoint['generator_state_dict'])
    discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
    generator_opt.load_state_dict(checkpoint['optimizerG_state_dict'])
    discriminator_opt.load_state_dict(checkpoint['optimizerD_state_dict'])
    gradient_p = checkpoint['gradient_p']
    G_losses = checkpoint['G_losses']
    D_losses = checkpoint['D_losses']
    mmd_list = checkpoint['mmd_list']
    series_list = checkpoint['series_list']
    #epoch_loaded = checkpoint['epoch']
    
    plt.plot(G_losses)
    plt.plot(D_losses)
    plt.plot(mmd_list)
    plt.plot(gradient_p)
    plt.show()
    print('Models loaded!')



wandb.watch([generator, discriminator], log='all', log_freq=80*1)
#wandb.watch(, log='all',  log_freq=80*2)

generator.train()
discriminator.train()

gradient_penalty = torch.ones(1).to(device)


input1 = torch.FloatTensor(sample_size, 2).to(device)
input2 = torch.FloatTensor(sample_size, 2).to(device)
interp_alpha = torch.FloatTensor(sample_size, 1).to(device)

fixed_noise = torch.FloatTensor(sample_size, 2).normal_(0, 1).to(device)
perturbation = torch.FloatTensor(sample_size,2).to(device)   


itera = -1

for epoch in tq.tqdm(range(num_epoch)):
    if epoch <= epoch_loaded: continue
    for n_batch, sample_data in enumerate(data_loader): # enumerate(tq.tqdm(data_loader)): #tq.tqdm(enumerate(data_loader)): 

      lr_g = lr_g * lr_decrease_factor
      lr_d = lr_d * lr_decrease_factor
    
      itera = itera + 1

      for p in discriminator.parameters():  # reset requires_grad
        p.requires_grad = True  # they are set to False below in netG update

      for d in range(D_rounds):
        
        #Train Discriminator on Fake Data
        discriminator.zero_grad()

        #Generating the noise and label data
        #noise_sample = Variable(noise(len(sample_data),seq_length))
        noise_sample = noise(len(sample_data), seq_length,noise_features )

        #Use this line if generator outputs hidden states: fake_data, (h_g_n,c_g_n) = generator.forward(noise_sample,h_g)
        #with torch.no_grad():
        fake_data = generator(noise_sample)#.detach()

        critic_out_fake = discriminator(fake_data)
        # tr = 0
        # for n in sample_data:
        #   if tr < 10:
        #     plt.plot(n)
        #     tr = 1 + tr
        # plt.show()
        #Train Discriminator on Real Data 
        real_data = sample_data.requires_grad_().to(device=device, dtype=torch.float)
        #real_data = sample_data.to(device=device, dtype=torch.float)


        critic_out_real  = discriminator(real_data)


        loss_discriminator = critic_out_fake.mean() - critic_out_real.mean()
        

        if lipschitz_constraint == 10:
            gradient_penalty = penalty_weight * wgan_gp_reg(discriminator, real_data, fake_data, center)
            #loss_discriminator = loss_discriminator #+  gradient_penalty

            #print(gradient_penalty)


        loss_discriminator.backward()
        discriminator_opt.step() #Updating the weights based on the predictions for both real and fake calculations.
          
          
    

      #Train Generator  
      for p in discriminator.parameters():  # reset requires_grad
        p.requires_grad = False  # they are set to False below in netG update
      #for g in range(G_rounds):


      generator.zero_grad()

      #noise_sample = Variable(noise(len(sample_data), seq_length))
      noise_sample = noise(len(sample_data), seq_length, noise_features)
      #Use this line if generator outputs hidden states: gen_fake_data, (h_g_n,c_g_n) = generator.forward(noise_sample,h_g)
      fake_data = generator(noise_sample)
      critic_out_fake = discriminator(fake_data)
      loss_generator = - critic_out_fake.mean()
      #loss_generator = torch.mean(critic_out_fake)
      loss_generator.backward() #loss_generator.backward(mone) #https://medium.com/@zhang_yang/the-gradient-argument-in-pytorchs-backward-function-explained-by-examples-68f266950c29
      #error_gen = loss_1(y_pred_gen,torch.ones([len(sample_data),1]).to(device))
      #error_gen.backward()
      generator_opt.step()
        
      gradient_p.append(gradient_penalty.item())
      G_losses.append(loss_generator.item())
      D_losses.append((loss_discriminator).item())

    wandb.log({"criticReal":critic_out_real.cpu(), "criticFake":critic_out_fake.cpu()  , "Critic": loss_discriminator.cpu(), "Generator": loss_generator.cpu(), "GradientPenalty": gradient_penalty.cpu()})

    with torch.no_grad():
        discriminator.eval()
        generator.eval()
        fake = generator(noise(len(sample_data), seq_length, noise_features)).detach().cpu()
        fd = discriminator(fake.to(device)).detach().cpu()                
        num_samples = 10
        fig = go.Figure()
        for i in range(num_samples):
            fig.add_trace(go.Scatter(x=np.arange(seq_length), y=fake[i].cpu(),
                mode='lines',
                name='s'+str(i)))
        wandb.log({'Epoch ' + str(epoch): fig})
        
        #real:
        if epoch == 1:
          fig = go.Figure()
          for i in range(num_samples):
              fig.add_trace(go.Scatter(x=np.arange(seq_length), y=real_data[i].cpu(),
                  mode='lines',
                  name='real'))
          wandb.log({'Real ': fig})
        #####################################################################################################################################
        avg_size = 500

        rnoise = torch.FloatTensor(avg_size, seq_length).uniform_(-1, 1)
        line = torch.zeros(avg_size, seq_length) 
        for i, s in enumerate(line): 
          line[i] = s + random.uniform(-1, 1)
        def sineWave1(size = 1, rand=1, amp=0, rad=0, phase=0, offsetY=0):
          res = torch.zeros(size, 50)
          for t in range(size):
            x = np.arange(0,20,0.4)
            count = 0
            fs = len(x)
            if rand:
              amp = random.uniform(0.1*1, 0.9*1)
              rad = random.uniform(5, 10)
              phase = random.uniform(-fs/0.5*np.pi,fs/0.5*np.pi)
              offsetY =  0# random.uniform(-1, 1) *1
          
            y = offsetY + amp*np.sin(((2*np.pi*rad*x)+phase)/fs) 
            if 0:
              for i, s in enumerate(y): 
                y[i] = s + random.uniform(-0.05, 0.001*i)
            y = torch.from_numpy(y).to(dtype=torch.float)
            res[t] = y

          return res.unsqueeze(2)

        sine = sineWave1(avg_size)

        fake = generator(noise(avg_size, seq_length, noise_features))
        real_out = discriminator(sine)
        rnoise_out = discriminator(rnoise)
        fake_out = discriminator(fake)
        line_out = discriminator(line)

        print(f'real input: {real_out.mean():.8f} real std: {real_out.std() }fake input: {fake_out.mean():.8f}  random input: {rnoise_out.mean():.8f} line input: {line_out.mean():.8f}')

        data = [real_out.detach().squeeze(1).numpy(), rnoise_out.detach().squeeze(1).numpy(), line_out.detach().squeeze(1).numpy(), fake_out.detach().squeeze(1).numpy()]
        group_labels = ['real', 'noise', 'line', 'generator']

        # Create distplot with custom bin_size
        fig = ff.create_distplot(data, group_labels, bin_size=math.pow(0.5/ math.log(avg_size),2))
        #fig.show()
        wandb.log({'CriticOut ' + str(epoch): fig})

                
        #####################################################################################################################################


        discriminator.train()
        generator.train()


        
                 

        series_list = np.append(series_list,fake[0].numpy().reshape((1,seq_length)),axis=0)

        #Saving the parameters of the model to file for each epoch
        if epoch % 5 == 0:
          saveModel(path, epoch)
          saveModel('/content/', 999)
          wandb.save('models.tar', base_path='/content')

                 

        series_list = np.append(series_list,fake[0].numpy().reshape((1,seq_length)),axis=0)

        #Saving the parameters of the model to file for each epoch
        saveModel(path, epoch)
        saveModel('/content/', 999)
        wandb.save('epoch999.tar', base_path='/content')

print('Training Complete!')
modelName = 'modelEpoch199.tar'
saveModel('/content/', 199)
wandb.save(modelName, base_path='/content')

print(f'Model {modelName} Saved!')


Using : cpu


real input: -0.07378540 real std: 0.8663672804832458fake input: -1.50510430  random input: -0.11205348 line input: 0.53075689
real input: -3.74167109 real std: 0.6623743772506714fake input: -5.43912888  random input: -3.77001047 line input: -3.88047409
real input: -0.90804911 real std: 0.8259062767028809fake input: -2.33670235  random input: -0.93922669 line input: -1.09440529
real input: -1.52961385 real std: 0.9349216222763062fake input: -3.36600304  random input: -1.48538995 line input: -1.79905427
real input: -1.20950925 real std: 0.9636860489845276fake input: -3.15275502  random input: -1.11510062 line input: -1.41923046
real input: -1.10475504 real std: 0.9295527338981628fake input: -2.96067858  random input: -1.11261451 line input: -1.41728020
real input: -0.88136047 real std: 0.9711174964904785fake input: -2.83292770  random input: -0.86421663 line input: -1.19618452
real input: -0.38411817 real std: 1.0026293992996216fake input: -2.36785913  random input: -0.40848726 line inpu

In [ ]:
saveModel('/content/', 500)
wandb.save('epoch500.tar', base_path='/content')


In [ ]:
num_samples = 9
num_classes = 40
df = pd.DataFrame(index=range(num_samples), columns=[str(x) for x in range(num_classes)])
for i in range(num_samples):
    df.iloc[i] = torch.randn(1, num_classes).numpy().round(2)

print(df)

In [ ]:
import os

torch.save({
        'generator_state_dict': generator.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
        'optimizerG_state_dict': generator_opt.state_dict(),
        'optimizerD_state_dict': discriminator_opt.state_dict()   
        }, 'wgan3.tar')
print('Model saved!')


In [ ]:
testNoise = noise(sample_size, int(seq_length*1), noise_features)
testOut = generator(testNoise)
for n in testOut:
  plt.plot(n.detach().cpu())
plt.show()

In [ ]:
#from mpl_toolkits.mplot3d import Axes3D

generator.eval()
discriminator.eval()

testNoise = noise(2, int(seq_length*1), noise_features)
testOut = generator(testNoise)

for n in testOut:
    plt.plot(n.detach().cpu())
plt.show()

# plt.show()
# plt.figure(figsize=(16,10))
# plt.plot(testOut[0].detach().cpu(), 'black') 
# plt.plot(testOut[0].detach().cpu() , 'bisque')
# plt.plot(testOut[1].detach().cpu() , 'aquamarine')
# plt.plot(testOut[2].detach().cpu() , 'lightsteelblue')
# plt.plot(testOut[3].detach().cpu() , 'lightcoral')
# plt.plot(testOut[4].detach().cpu() , 'cornsilk')
# plt.plot(testOut[5].detach().cpu() , 'thistle')
# plt.plot(testOut[6].detach().cpu() , 'peachpuff')
# plt.plot(testOut[7].detach().cpu() , 'powderblue')
# plt.show()


r1u =  0 # delta
r1l = -0 # delta
r2u =  0 # amplitude
r2l = -0 # amplitude
r3u =  0 # freq (+ amplitude)
r3l = -0 # freq (+ amplitude)
r4u =  0 # freq (+ amplitude)
r4l = -0 # freq (+ amplitude)

for b in range(2):
  r1 = random.uniform(r1u,r1l)
  r2 = random.uniform(r2u,r2l)
  r3 = random.uniform(r3u,r3l)    #-> noise_features = 3
  r4 = random.uniform(r4u,r4l)
  #r4 = random.uniform(-1,1)   #-> noise_features = 4 ...
  for i in range(seq_length):
    testNoise[b][i][0] = r1  
    testNoise[b][i][1] = r2 
    testNoise[b][i][2] = r3 
  #  testNoise[b][i][3] = r4
    #noise[b][i][3] = r4
    
# fig = plt.figure(constrained_layout=True)
# ax = fig.add_subplot( projection='3d')  
# for b in range(sample_size):
#     ax.scatter(testNoise[b][0][0].cpu(), testNoise[b][0][1].cpu(), testNoise[b][0][2].cpu(), marker=".")

# ax.set_xlabel('Freq')
# ax.set_ylabel('Ampl')
# ax.set_zlabel('delta')
# plt.show()
for i in range(seq_length):
    testNoise[0][i][0] = r1  
    testNoise[0][i][1] = r2 
    testNoise[0][i][2] = r3 
#    testNoise[0][i][3] = r4
    testNoise[1][i][0] = r1 + 0.
    testNoise[1][i][1] = r2 + 0.
    testNoise[1][i][2] = r3 + 0.9  # Yoffset
#    testNoise[0][i][3] = r4 + -0.
testOut = generator(testNoise)

plt.show()
plt.figure(figsize=(16,10))
plt.plot(testOut[0].detach().cpu(), 'black') 
plt.plot(testOut[1].detach().cpu() , 'red')
plt.show()

In [ ]:
# checkpoint = torch.load('/content/epoch9999.tar')#torch.load(loadPath)
# generator.load_state_dict(checkpoint['generator_state_dict'])
# discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
# generator_opt.load_state_dict(checkpoint['optimizerG_state_dict'])
# discriminator_opt.load_state_dict(checkpoint['optimizerD_state_dict'])
# print('Models loaded!')


In [ ]:
#rnoise = torch.ones( (4, seq_length, noise_features)).to(device)
avg_size = 1000

rnoise = torch.FloatTensor(avg_size, seq_length).uniform_(-1, 1)
# for i, s in enumerate(rnoise): 
#   rnoise[i] = s * random.uniform(-2, 2)
rnoise_out = discriminator(rnoise)

line = torch.zeros(avg_size, seq_length) 
for i, s in enumerate(line): 
  line[i] = s + random.uniform(-1, 1)

line_out = discriminator(line)

#print(f'random input: {rnoise_out}')
#print(f'line input: {line_out}')
def sineWave1(size = 1, rand=1, amp=0, rad=0, phase=0, offsetY=0):
  res = torch.zeros(size, 50)
  for t in range(size):
    #a = np.arange(0.1,0.9,0.02)
    x = np.arange(0,20,0.4)
    #r = np.arange(2,6.1,0.1)
    count = 0
    fs = len(x)
    #y = np.zeros((1,len(x)))
    #y = [50]
    if rand:
      amp = random.uniform(0.1*1, 0.9*1)
      rad = random.uniform(5, 10)
      phase = random.uniform(-fs/0.5*np.pi,fs/0.5*np.pi)
      offsetY =  0# random.uniform(-1, 1) *1
  
    y = offsetY + amp*np.sin(((2*np.pi*rad*x)+phase)/fs) 
    if 0:
      for i, s in enumerate(y): 
        y[i] = s + random.uniform(-0.05, 0.001*i)
    y = torch.from_numpy(y).to(dtype=torch.float)
    res[t] = y

  return res.unsqueeze(2)

sine = sineWave1(avg_size)

fake = generator(noise(avg_size, seq_length, noise_features))
fake_out = discriminator(fake)
#print(f'sine: {sine.shape}')
# plt.plot(sine[0])
# plt.plot(sine[1])
# plt.plot(rnoise[0])
# plt.plot(line[0])
# plt.show()
#sine_tensor = torch.from_numpy(sine).unsqueeze(0).unsqueeze(2).to(device=device, dtype=torch.float)

real_out = discriminator(sine)
#print(f'real input: {real_out}')
print(f'real input: {real_out.mean():.8f} real std: {real_out.std() }fake input: {fake_out.mean():.8f}  random input: {rnoise_out.mean():.8f} line input: {line_out.mean():.8f}')

data = [real_out.detach().squeeze(1).numpy(), rnoise_out.detach().squeeze(1).numpy(), line_out.detach().squeeze(1).numpy(), fake_out.detach().squeeze(1).numpy()]
# fig, ax = plt.subplots()
# fig.set_size_inches(18.5, 10.5)
# bp1 = ax.boxplot(data[0], positions=[1], notch=True, widths=0.35, 
#                  patch_artist=True, boxprops=dict(facecolor="C0"))
# bp2 = ax.boxplot(data[1], positions=[2], notch=True, widths=0.35, 
#                  patch_artist=True, boxprops=dict(facecolor="C2"))
# bp3 = ax.boxplot(data[2], positions=[3], notch=True, widths=0.35, 
#                  patch_artist=True, boxprops=dict(facecolor="C3"))
# ax.legend([bp1["boxes"][0], bp2["boxes"][0], bp3["boxes"][0]], ['real', 'noise uniform(-1, 1)', 'line uniform(-1, 1)'])



hist_data = [data[0], data[1], data[2], data[3]]

group_labels = ['real', 'noise', 'line', 'generator']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=math.pow(0.5/ math.log(avg_size),2))
fig.show()

In [ ]:
#wandb.log({'RealvsNoise ' + str(avg_size): fig})

In [ ]:


#data = [real_out.detach().squeeze(1).numpy(), rnoise_out.detach().squeeze(1).numpy(), line_out.detach().squeeze(1).numpy(), fake_out.detach().squeeze(1).numpy()]
fig, ax = plt.subplots()
fig.set_size_inches(18.5, 10.5)
bp1 = ax.boxplot(data[0], positions=[1], notch=True, widths=0.35, 
                 patch_artist=True, boxprops=dict(facecolor="C0"))
bp2 = ax.boxplot(data[1], positions=[2], notch=True, widths=0.35, 
                 patch_artist=True, boxprops=dict(facecolor="C2"))
bp3 = ax.boxplot(data[2], positions=[3], notch=True, widths=0.35, 
                 patch_artist=True, boxprops=dict(facecolor="C3"))
ax.legend([bp1["boxes"][0], bp2["boxes"][0], bp3["boxes"][0]], ['real', 'noise uniform(-1, 1)', 'line uniform(-1, 1)'])
# ax.annotate('outlier', xy=(2, 1), xytext=(1.5, 1),
#             arrowprops=dict(facecolor='black', shrink=0.05))

# ax.text(1.5, 0.8, 'Boxplot of critic output over 10.000 samples each', style='italic',
#         bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10})

In [ ]:
#data = [real_out.detach().squeeze(1).numpy(), rnoise_out.detach().squeeze(1).numpy(), line_out.detach().squeeze(1).numpy(), fake_out.detach().squeeze(1).numpy()]



# Group data together
hist_data = [data[0], data[1], data[2], data[3]]

group_labels = ['real', 'noise', 'line', 'generator']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[100/avg_size, 100/avg_size, 100/avg_size, 100/avg_size])
fig.show()
#wandb.log({'RealvsNoise ' + str(epoch): fig})

In [ ]:
diffSine = torch.zeros(500, 50, 1)

for i in range(500):
  diffSine[i] = sineWave1(size=1, rand= 1, amp=0.55, rad=5, phase=2, offsetY=0) 
  plt.plot(diffSine[i])
plt.show()


diffSine_out =  discriminator(diffSine)
print('-2.45 bis -2.9')
print(diffSine_out)

for i, s in enumerate(diffSine_out.squeeze(0)): 
  #line[i] = s + random.uniform(0.3, 1)
 if diffSine_out[i] < -2.9:
    plt.plot(diffSine[i])
    print(f'dangerous: {i}')

  # if diffSine_out[i] > -2.45:
  # #  if real_out[i] > -3.35:
  #     plt.plot(diffSine[i])
  #     print(f'maybe ok: {i}')
 else:
    print(f'good : {i}')
plt.show()




In [ ]:
for i, s in enumerate(real_out.squeeze(0)): 
  #line[i] = s + random.uniform(0.3, 1)
  if real_out[i] > -3.65 and real_out[i] < -3.645:
#  if real_out[i] > -3.35:
    plt.plot(sine[i])


plt.show()

In [ ]:
# Averaged over batchsize of 100.000
# base-line real: ~3.12           stdev: 0.69
# random: ~3.89
# 2-line: ~4.89
# 1-line: ~4.18

# higher freq: 3.26
# amp * 10: 3.58
# amp * 2: 3.06
# offsetY * 10: 3.25
# offsetY * 2: 2.99





